In [1]:
import enchant
import pickle
import sys
from Levenshtein.StringMatcher import StringMatcher
import pyprind

sys.path.insert(1, '../../dataset')
import dataset_map_entry

** 1. Method for loading dataset **

In [2]:
def load_dataset():
    input_dataset = open('../../dataset/output/map_final.pkl', 'rb')
    sys.modules['dataset_map_entry'] = dataset_map_entry
    dataset = pickle.load(input_dataset)
    input_dataset.close()

    return dataset

dataset = load_dataset()

** 2. Creating dictionary (and everything else that is necessary) for storing counts of 'out of dictionary words' with respect to the total number of words. **

In [3]:
# dictionary with english words
d = enchant.Dict("en_US")
# dictionary with spanish words (because they may occurr in english tweets)
d_sp = enchant.Dict("es")
# Tool for calculating Levenshtein word distance
sm = StringMatcher()
# List with out of dictionary words
out_of_dict_words = set()
# List with words in dictionary
in_dict_words = set()
# Levensthein word distance ratio threshold
WORD_DIST_RATIO = 0.6
#load brands
brands = set(line.lower().strip() for line in open('brands.txt'))
# dictionary with (user, (no_of_out_of_dict_words, total_no_of_words)) pair
user_map = {}

** 3. Storing pairs (no_of_out_dict_words, total_no_of_words) in dictionary **

In [4]:
pBar = pyprind.ProgBar(len(dataset))
for user in dataset:
    for tweet in dataset[user].tweets:
        for word in tweet:
            if word.replace('\'','').replace('-','').isalpha() and not d.check(word) \
             and not d_sp.check(word) and (word not in brands) and ('URL' not in word) and ('NUMBER' not in word):
                sm.set_seq1(seq1=word)
                founded = False
                for suggestion in d.suggest(word):
                    sm.set_seq2(seq2=suggestion)
                    if sm.ratio > WORD_DIST_RATIO:
                        out_of_dict_words.add(word)
                        founded = True
                        break
                if not founded:
                    in_dict_words.add(word)
            else:
                in_dict_words.add(word)     
    user_map[user] = (len(out_of_dict_words), len(out_of_dict_words) + len(in_dict_words))
    out_of_dict_words.clear()
    in_dict_words.clear()
    pBar.update()            
    
                    

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:03:37


** 4. Storing dictionary in a file **

In [5]:
output_dir = open('../pkls/out_of_dict_map.pkl', 'wb')
pickle.dump(user_map, output_dir)
output_dir.close()